# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [24]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [4]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,boguchany,58.3814,97.4531,270.85,89,96,5.36,RU,1704766568
1,1,at-bashi,41.1702,75.8106,259.17,69,26,1.11,KG,1704766667
2,2,puerto ayora,-0.7393,-90.3518,298.46,96,100,1.34,EC,1704766667
3,3,isafjordur,66.0755,-23.1240,278.12,90,100,5.71,IS,1704766668
4,4,keetmanshoop,-26.5833,18.1333,295.50,33,17,2.68,NaN,1704766668


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
map_plot=city_data_df.hvplot.points("Lng","Lat",geo=True,tiles="OSM",frame_width=700,frame_height=500,size=city_data_df["Humidity"],color=city_data_df["City"])

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (_color,_size)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [14]:
# Narrow down cities that fit criteria and drop any results with null values
cloud_df=city_data_df.loc[city_data_df["Cloudiness"]<=25]
ideal_df=cloud_df.loc[cloud_df["Humidity"]<=25]
# Drop any rows with null values
# YOUR CODE HERE
ideal_df.dropna()
# Display sample data
ideal_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
160,160,san patricio,28.0170,-97.5169,291.57,24,0,9.89,US,1704766706
166,166,karratha,-20.7377,116.8463,310.24,21,3,5.55,AU,1704766707
179,179,konkwesso,10.8469,4.0984,295.02,12,10,2.02,NG,1704766711
207,207,tidjikja,18.5564,-11.4271,292.70,13,22,4.64,MR,1704766718
231,231,kidal,18.4411,1.4078,293.22,14,0,4.70,ML,1704766724
249,249,tamanrasset,22.7850,5.5228,284.10,16,0,3.60,DZ,1704766731
260,260,gerash,27.6652,54.1371,274.23,22,0,0.46,IR,1704766733
266,266,jikamshi,12.1733,7.7742,290.60,13,1,3.84,NG,1704766734
307,307,rikaze,29.2500,88.8833,270.26,16,0,0.11,CN,1704766761
308,308,gummi,12.1436,5.1081,293.94,14,1,3.94,NG,1704766762


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df=ideal_df[["City","Country","Lat","Lng","Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]={}

# Display sample data
hotel_df

C:\Users\jacqu\AppData\Local\Temp\ipykernel_24048\3882673099.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]={}


,City,Country,Lat,Lng,Humidity,Hotel Name
160,san patricio,US,28.0170,-97.5169,24,NaN
166,karratha,AU,-20.7377,116.8463,21,NaN
179,konkwesso,NG,10.8469,4.0984,12,NaN
207,tidjikja,MR,18.5564,-11.4271,13,NaN
231,kidal,ML,18.4411,1.4078,14,NaN
249,tamanrasset,DZ,22.7850,5.5228,16,NaN
260,gerash,IR,27.6652,54.1371,22,NaN
266,jikamshi,NG,12.1733,7.7742,13,NaN
307,rikaze,CN,29.2500,88.8833,16,NaN
308,gummi,NG,12.1436,5.1081,14,NaN


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [92]:
# Set parameters to search for a hotel
radius = 10000
categories = ["accommodation.hotel"]
params = {"categories": categories,
"apiKey" : geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
san patricio - nearest hotel: No hotel found
karratha - nearest hotel: Karratha International Hotel
konkwesso - nearest hotel: No hotel found
tidjikja - nearest hotel: No hotel found
kidal - nearest hotel: No hotel found
tamanrasset - nearest hotel: فندق الأمان
gerash - nearest hotel: هتل آبشار
jikamshi - nearest hotel: No hotel found
rikaze - nearest hotel: Gesar Hotel
gummi - nearest hotel: No hotel found
cutral-co - nearest hotel: Hotel Karim
abocho - nearest hotel: No hotel found
plaza huincul - nearest hotel: Antu Malal Aparts & Suites
douentza - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
160,san patricio,US,28.0170,-97.5169,24,No hotel found
166,karratha,AU,-20.7377,116.8463,21,Karratha International Hotel
179,konkwesso,NG,10.8469,4.0984,12,No hotel found
207,tidjikja,MR,18.5564,-11.4271,13,No hotel found
231,kidal,ML,18.4411,1.4078,14,No hotel found
249,tamanrasset,DZ,22.7850,5.5228,16,فندق الأمان
260,gerash,IR,27.6652,54.1371,22,هتل آبشار
266,jikamshi,NG,12.1733,7.7742,13,No hotel found
307,rikaze,CN,29.2500,88.8833,16,Gesar Hotel
308,gummi,NG,12.1436,5.1081,14,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [96]:
%%capture --no-display

# Configure the map plot
hotel_plot=hotel_df.hvplot.points("Lng","Lat",geo=True,tiles="OSM",frame_width=700,frame_height=500,size="Humidity",color="City",hover_cols=["Hotel Name","Country"])

# Display the map
hotel_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)